Dati i tweet in database, salva gli utenti:
- Regolari
- Attivi
- Regolari e Attivi

Inoltre, salva texted_tweet e retweet di utenti Regolari e Attivi

In [4]:
import sys



PATH = "../../.."
sys.path.append(f'{PATH}/')

from src.utils.pyspark_init import set_pySpark
from src.utils.globalVariable import *
from src.dataProcessing.tweet.Tweet import Tweet
from src.dataProcessing.active_and_regular_user.RegularAndActiveUsers import RegularAndActiveUser
from src.utils.slidingWindow import get_period_list, utc_to_string
from src.utils.globalVariable import REGULAR_ACTIVE_USER_DIR, TWEET_DATABASE_DIR
import pyspark.pandas as ps
from src.utils.globalVariable import USER_DATABASE_DIR

set_pySpark()

Rec

Recupera gli utenti regolari e attivi periodo per periodo

In [ ]:
period_list = get_period_list()
for first_period, second_period in period_list:
    first_period_string = utc_to_string(first_period)
    second_period_string = utc_to_string(second_period)

    regularActiveUsers = RegularAndActiveUser(f"{PATH}/{TWEET_DATABASE_DIR}", first_period, second_period)
    regularActiveUsers = regularActiveUsers.get_regularAndActiveUser()
    regularActiveUsers.to_parquet(f"{PATH}/{REGULAR_ACTIVE_USER_DIR}/temp/{first_period_string}-{second_period_string}/user_id")

In [5]:
period_list = get_period_list()
regularActiveUsers = ps.read_parquet(f"{PATH}/{REGULAR_ACTIVE_USER_DIR}temp/*/user_id")
numPeriod = len(period_list)
#Selezione l'intersezione degli utenti in tutti i periodi
regularActiveUsers = ps.sql("SELECT user_id, COUNT(*) AS count "
                            "FROM {regularActiveUsers} "
                            "GROUP BY user_id "
                            "HAVING COUNT(*) = {numPeriod}",
                            regularActiveUsers=regularActiveUsers, numPeriod=numPeriod)

user = ps.read_parquet(f"{PATH}/{USER_DATABASE_DIR}")
user = ps.sql("SELECT * "
              "FROM {user} "
              "WHERE id IN (SELECT user_id FROM {regularActiveUsers})",
              user=user, regularActiveUsers=regularActiveUsers)

#Seleziona gli utenti che hanno creato il profilo almeno 6 mesi prima dell'inizio dell'analisi
second_6month = 15778800
start_analysis = period_list[0][0]
user = ps.sql("SELECT * "
              "FROM {user} "
              "WHERE created_at_utc <= {start_analysis}-{second_6month} "
              "ORDER BY created_at_utc DESC ",
              user=user, start_analysis=start_analysis, second_6month=second_6month)

user.to_parquet(f"{PATH}/{REGULAR_ACTIVE_USER_DIR}/intersection/user")

/home/kcekini/progetti/Twitter/.venv/lib/python3.9/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_parquet`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
